In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q trl
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import datasets
from datasets import load_dataset

In [57]:
dataset = load_dataset("Mreeb/Dermatology-Question-Answer-Dataset-For-Fine-Tuning")

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [21]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_id = "epfl-llm/meditron-7b"


model = AutoModelForCausalLM.from_pretrained(
  model_id,
  attn_implementation="flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
  torch_dtype="auto",
  use_cache=False, # set to False as we're going to use gradient checkpointing
  quantization_config=bnb_config,
  device_map={"":0}
)

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/262M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [35]:
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

tokenizer_config.json:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [58]:
data = dataset["train"]

In [55]:
def generate_prompt(data_point):
    """
    Generate input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenized prompt
    """
    # Create a text with just instruction and response
    # text = 'Below is an instruction that describes a task. Write a response that ' \
    #         'appropriately completes the request.\n\n'
    # text += f'### Instruction:\n{data_point["prompt"]}\n\n'
    # text += f'### Response:\n{data_point["response"]}'

    text = 'Suppose you are a Dermotology professional. Answer the below question.\n\n</s>'
    text += f'### Question:\n{data_point["prompt"]}\n\n</s>'
    text += f'### Answer:\n{data_point["response"]}'



    return text

In [60]:
text_column = [generate_prompt(data_point) for data_point in data]

In [61]:
data = data.add_column('text', text_column)

In [62]:
data = data.remove_columns(['prompt_word_count', 'response_word_count'])

In [63]:
# from multiprocessing import cpu_count
# cpu_count()
%time
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


Map:   0%|          | 0/1460 [00:00<?, ? examples/s]

In [64]:
data

Dataset({
    features: ['prompt', 'response', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1460
})

In [65]:
data = data.shuffle(seed=1234)

In [66]:
# prompt: split data in train and test
data = data.train_test_split(test_size=0.1)

In [67]:
train_data = data["train"]
test_data = data["test"]

In [51]:
def inference(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  Suppose you are a Dermotology professional. Answer the below question.</s>
  ### Question:
  {query}
  </s>
  ### Answer:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=150, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [68]:
result = inference(test_data[0]["prompt"], model, tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
  Suppose you are a Dermotology professional. Answer the below question.
  ### Question:
  Are there any advanced therapies available for alopecia areata?

  ### Answer:
  </s>    Yes, there are advanced therapies available for alopecia areata.   They include oral medication (corticosteroid), topical medications (corticosteroids, calcineurin inhibitors, retinoid therapy), phototherapy, light therapy, scalp-massage techniques (Migrocomb), local therapies (botulinum toxin injections), laser therapy and surgical therapy.
 
 
  In conclusion, advanced therapies are available for alopecia areata.
  
 
  

  
  
  #### Disclaimer
  #### 
  This


In [69]:
result = inference(test_data[1]["prompt"], model, tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
  Suppose you are a Dermotology professional. Answer the below question.
  ### Question:
  What is the cause of eczema, its symptoms, and how is it treated?

  ### Answer:
  </s>  
  **Dermatologists**

The cause is unknown. The signs and symptoms include itching that is sometimes relieved by scratching. There are often blisters.

The first step in the treatment is to prevent the itch. There are prescription medicines that do this. Then there are topical medications. These are applied to the affected area directly. A prescription product is available, also. Finally, corticosteroid shots may be needed to denervate the area. This stops the itch from spreading.





<EOD> Impairment of Vestibular Response in Chronic Sinusitis and Adenotonsil


In [70]:
result = inference(test_data[2]["prompt"], model, tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
  Suppose you are a Dermotology professional. Answer the below question.
  ### Question:
  What are the causes, symptoms, and treatments for psoriasis?

  ### Answer:
  </s>
Dermatology is a branch of the medicine that specializes in the treatment of skin conditions and allergies. Dermatology services include removing unsightly skin warts and cysts. It can also perform a wide range of treatments for psoriasis, one of the most common skin conditions. One of the main symptoms of psoriasis is itching, which affects the scalp, face, elbows, and knees. To treat psoriasis, it is recommended to take medication, and if you are allergic, you need to avoid medicines that have chemical ingredients and dyes.
> Try it yourself. See if the code works


In [71]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32017, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): L

In [72]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,                                   # Number of quantization levels
    lora_alpha=32,                         # Hyperparameter for LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # Modules to apply LoRA to
    lora_dropout=0.05,                     # Dropout probability
    bias="none",                           # Type of bias
    task_type="CAUSAL_LM"                  # Task type (in this case, Causal Language Modeling)
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [73]:
model.print_trainable_parameters()

trainable params: 16,777,216 || all params: 6,755,332,096 || trainable%: 0.24835516243434141


In [74]:
from trl import SFTTrainer
import transformers
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="text",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        fp16=True,
        warmup_steps=0.03,
        max_steps=10,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1314 [00:00<?, ? examples/s]

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [75]:
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


Step,Training Loss
1,1.445500
2,1.506300
3,1.235200
4,1.250500
5,1.257000
6,1.305500
7,1.284800
8,1.352900
9,1.098500
10,1.181200


TrainOutput(global_step=10, training_loss=1.2917307257652282, metrics={'train_runtime': 19.6847, 'train_samples_per_second': 2.032, 'train_steps_per_second': 0.508, 'total_flos': 367046393610240.0, 'train_loss': 1.2917307257652282, 'epoch': 0.03})

In [76]:
model.push_to_hub("medron_7B_Dermotologist")
tokenizer.push_to_hub("medron_7B_Dermotologist")

adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ubaidbhat/medron_7B_Dermotologist/commit/c659fd24fe956cda2064fb0781746ab86fb63655', commit_message='Upload tokenizer', commit_description='', oid='c659fd24fe956cda2064fb0781746ab86fb63655', pr_url=None, pr_revision=None, pr_num=None)

In [77]:
from peft import PeftConfig, PeftModel
peft_model_id = "Ubaidbhat/medron_7B_Dermotologist"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

# merged_model = model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

In [78]:
result = inference(test_data[0]["prompt"], model, tokenizer)
print(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<s> 
  Suppose you are a Dermotology professional. Answer the below question.
  ### Question:
  Are there any advanced therapies available for alopecia areata?

  ### Answer:
  
There are currently no FDA-approved treatments available for alopecia areata, but research and development of promising new therapies is ongoing. Previous studies have shown that intradermal injections of autologous hair matrix cells into lesions may have some effect on regrowth of hair, but the results are variable and more research is needed to determine the potential benefits of this treatment. In addition, there are reports of treatments with high-dose vitamin A, but currently there is no evidence to support its efficacy and no standardized protocol for dosing.
Dermatologists usually make therapeutic decisions based on several factors, including the size of the


Before Fine-tunning:
<s>
  Suppose you are a Dermotology professional. Answer the below question.
  ### Question:
  Are there any advanced therapies available for alopecia areata?

  ### Answer:
  </s>    Yes, there are advanced therapies available for alopecia areata.   They include oral medication (corticosteroid), topical medications (corticosteroids, calcineurin inhibitors, retinoid therapy), phototherapy, light therapy, scalp-massage techniques (Migrocomb), local therapies (botulinum toxin injections), laser therapy and surgical therapy.


  In conclusion, advanced therapies are available for alopecia areata.
  

  

  
  
  #### Disclaimer
  ####
  This
  

In [81]:
result = inference(test_data[2]["prompt"], model, tokenizer)
print(result)

<s> 
  Suppose you are a Dermotology professional. Answer the below question.
  ### Question:
  What are the causes, symptoms, and treatments for psoriasis?

  ### Answer:
  1. Psoriasis is a skin condition caused by immune system dysfunction, with genetic predisposition, that presents dry, raised, scaly, and itchy patches on the skin. It affects almost 7% of the world's population and can be inherited or can occur due to stress, trauma, or injuries. Psoriasis affects the skin, scalp, nails, and even mucous membranes. The main symptoms are itching, burning, pain, swelling, and stiffness of joints, in the case of psoriatic arthritis. Psoriasis can be cured with regular treatment and maintenance. The


<s>
  Suppose you are a Dermotology professional. Answer the below question.
  ### Question:
  What are the causes, symptoms, and treatments for psoriasis?

  ### Answer:
  </s>
Dermatology is a branch of the medicine that specializes in the treatment of skin conditions and allergies. Dermatology services include removing unsightly skin warts and cysts. It can also perform a wide range of treatments for psoriasis, one of the most common skin conditions. One of the main symptoms of psoriasis is itching, which affects the scalp, face, elbows, and knees. To treat psoriasis, it is recommended to take medication, and if you are allergic, you need to avoid medicines that have chemical ingredients and dyes.
> Try it yourself. See if the code works